# Nettoyage des données eBay pour Smart Watches

Ce notebook présente un pipeline complet de nettoyage des données pour les montres connectées extraites d'eBay.

**Table des matières :**
1. Importation des bibliothèques
2. Définition des chemins d'accès aux données
3. Définition des fonctions de nettoyage :
   - Nettoyage du prix (`clean_price`)
   - Nettoyage de la taille du boîtier (`clean_case_size`)
   - Nettoyage de la capacité de la batterie (`clean_battery_capacity`)
   - Nettoyage de la marque (`clean_brand`)
   - Nettoyage du modèle (`clean_model`)
   - Nettoyage du système d'exploitation (`clean_os`)
   - Nettoyage de la capacité de stockage (`clean_storage_capacity`)
   - Nettoyage du titre (`clean_title`)
   - Suppression des doublons (`remove_duplicates`)
4. Fonction principale de nettoyage des données
5. Traitement des fichiers CSV et sauvegarde des données nettoyées


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.impute import SimpleImputer
from fuzzywuzzy import process
from unidecode import unidecode
from pathlib import Path
import re


## Définition des chemins d'accès aux données

Nous définissons ici les chemins relatifs pour accéder aux données brutes et aux données nettoyées.
**Note :** Dans un notebook Jupyter, la variable `__file__` n'est pas définie. Nous utilisons donc `Path.cwd()` pour définir le répertoire de base.


In [ ]:
# Définir les chemins en utilisant pathlib
BASE_DIR = Path.cwd()  # Utilisation du répertoire courant comme base
RAW_DATA_DIR_EBAY = BASE_DIR / 'data' / 'raw' / 'ebay' / 'smart_watches'
CLEANED_DATA_DIR_EBAY = BASE_DIR / 'data' / 'cleaned' / 'ebay' / 'smart_watches'

# Créer le dossier pour les données nettoyées s'il n'existe pas
CLEANED_DATA_DIR_EBAY.mkdir(parents=True, exist_ok=True)

# Afficher les chemins pour vérification
print(f"Base directory: {BASE_DIR}")
print(f"Raw data directory (eBay): {RAW_DATA_DIR_EBAY}")
print(f"Cleaned data directory (eBay): {CLEANED_DATA_DIR_EBAY}")

# Vérifier que le dossier des données brutes existe
if not RAW_DATA_DIR_EBAY.exists():
    raise FileNotFoundError(f"Raw data directory not found: {RAW_DATA_DIR_EBAY}")


### Fonction `clean_price`

**Description :**
La fonction `clean_price` prend une valeur de prix (qui peut contenir des séparateurs de milliers ou des symboles de devise) et la transforme en une valeur numérique.

**Étapes :**
1. Si la valeur est manquante, retourne `np.nan`.
2. Convertit la valeur en chaîne de caractères et supprime les caractères indésirables (par exemple, `,`, `$`, `GBP`, etc.).
3. Essaye de convertir la chaîne nettoyée en nombre flottant.
4. En cas d'erreur de conversion, retourne `np.nan`.


In [ ]:
def clean_price(price):
    if pd.isna(price):
        return np.nan
    # Supprimer les caractères indésirables
    price = str(price).replace(',', '').replace('$', '').replace('GBP', '').replace('US', '') \
                     .replace('/ea', '').replace('AU', '').replace('EUR', '').strip()
    try:
        return float(price)
    except ValueError:
        return np.nan

# Exemple d'utilisation :
print("Exemple clean_price:", clean_price("$1,299.99"))


### Fonction `clean_case_size`

**Description :**
La fonction `clean_case_size` nettoie la donnée relative à la taille du boîtier des montres.

**Étapes :**
1. Si la valeur est manquante ou égale à `"Does not apply"`, retourne `np.nan`.
2. Retire l'unité `'mm'` et sépare les valeurs (si plusieurs tailles sont indiquées).
3. Convertit les valeurs en float et calcule la moyenne.
4. Retourne la moyenne arrondie à 2 décimales.


In [ ]:
def clean_case_size(case_size):
    if pd.isna(case_size) or case_size == 'Does not apply':
        return np.nan
    # Extraire les tailles en retirant 'mm' et séparer par des virgules
    sizes = [float(s) for s in str(case_size).replace('mm', '').split(',') if s.strip().replace('.', '', 1).isdigit()]
    return round(np.mean(sizes), 2) if sizes else np.nan

# Exemple d'utilisation :
print("Exemple clean_case_size:", clean_case_size("42mm, 44mm"))


### Fonction `clean_battery_capacity`

**Description :**
La fonction `clean_battery_capacity` nettoie le champ de la capacité de la batterie.

**Étapes :**
1. Si la valeur est manquante, retourne `np.nan`.
2. Retire l'unité `'mAh'`.
3. Convertit la valeur nettoyée en nombre (après arrondi) de type entier.
4. En cas d'erreur de conversion, retourne `np.nan`.


In [ ]:
def clean_battery_capacity(capacity):
    if pd.isna(capacity):
        return np.nan
    capacity = str(capacity).replace('mAh', '').strip()
    try:
        return int(round(float(capacity)))
    except ValueError:
        return np.nan

# Exemple d'utilisation :
print("Exemple clean_battery_capacity:", clean_battery_capacity("350mAh"))


### Fonction `clean_brand`

**Description :**
La fonction `clean_brand` nettoie le champ de la marque en s’appuyant sur :
- Une liste prédéfinie de marques reconnues.
- La bibliothèque `fuzzywuzzy` pour obtenir une correspondance approximative.

**Étapes :**
1. Si la marque est manquante ou égale à `"Does not apply"`, recherche dans le titre si une marque connue y est présente.
2. Si aucune correspondance n’est trouvée, retourne `"Unknown"`.
3. Sinon, utilise `process.extractOne` pour trouver la meilleure correspondance avec un score minimum de 80.


In [ ]:
# Liste des marques reconnues
brands = ['Google', 'Apple', 'Samsung', 'Xiaomi', 'Fitbit', 'Garmin',
          'Apple', 'Samsung', 'Huawei', 'IOWODO', 'iPhone', 'Pebble', 'TOZO', 'Fossil', 'Amazfit', 'Ticwatch', 'Mobvoi',
          'Verizon', 'COLMI', 'AICase', 'Haylou', 'HUAWEI', 'Honor', 'Xiaomi', 'Garmin', 'Fitbit', 'Fossil',
          'Withings', 'Nothing', 'T-Mobile']

def clean_brand(brand, title):
    if pd.isna(brand) or brand == 'Does not apply':
        for b in brands:
            if b.lower() in title.lower():
                return b
        return 'Unknown'
    else:
        match = process.extractOne(unidecode(str(brand)), brands)
        return match[0] if match and match[1] >= 80 else 'Unknown'

# Exemple d'utilisation :
print("Exemple clean_brand:", clean_brand(None, "Apple Watch Series 6"))


### Fonction `clean_model`

**Description :**
La fonction `clean_model` nettoie le champ du modèle.

**Étapes :**
1. Si le modèle est manquant ou vaut `"Does not apply"`, retourne `"Unknown"`.
2. Utilise `process.extractOne` pour trouver la meilleure correspondance parmi une liste de modèles existants avec un score minimum de 80.

> **Remarque :**
> La liste des modèles sera extraite du DataFrame dans la fonction principale.


In [ ]:
def clean_model(model, models_list):
    if pd.isna(model) or model == 'Does not apply':
        return 'Unknown'
    match = process.extractOne(unidecode(str(model)), models_list)
    return match[0] if match and match[1] >= 80 else 'Unknown'

# Exemple d'utilisation :
example_models = ['Series 6', 'Series 5', 'SE']
print("Exemple clean_model:", clean_model("Series 6", example_models))


### Fonction `clean_os`

**Description :**
La fonction `clean_os` nettoie le champ du système d’exploitation en le comparant à une liste de systèmes connus.

**Étapes :**
1. Si la valeur est manquante, retourne `"Unknown"`.
2. Utilise `process.extractOne` pour trouver la correspondance la plus proche dans la liste avec un score minimum de 80.


In [ ]:
# Liste des systèmes d'exploitation connus
os_list = ['Wear OS', 'Android Wear', 'Apple Watch OS', 'Tizen', 'LiteOS', 'Moto Watch OS', 'Pebble OS']

def clean_os(os_value):
    if pd.isna(os_value):
        return 'Unknown'
    match = process.extractOne(unidecode(str(os_value)), os_list)
    return match[0] if match and match[1] >= 80 else 'Unknown'

# Exemple d'utilisation :
print("Exemple clean_os:", clean_os("Android Wear"))


### Fonction `clean_storage_capacity`

**Description :**
La fonction `clean_storage_capacity` nettoie le champ de la capacité de stockage en gérant différentes unités.

**Étapes :**
1. Si la valeur est manquante, retourne `np.nan`.
2. Supprime les unités `"GB"` ou `"MB"`.
3. Si l'unité est en MB, convertit la valeur en GB.
4. Tente de convertir la chaîne en float ; en cas d'erreur, retourne `np.nan`.


In [ ]:
def clean_storage_capacity(storage):
    if pd.isna(storage):
        return np.nan
    storage_str = str(storage)
    # Déterminer l'unité et nettoyer la chaîne
    if 'MB' in storage_str:
        try:
            return float(storage_str.replace('MB', '').strip()) / 1024
        except ValueError:
            return np.nan
    else:
        try:
            return float(storage_str.replace('GB', '').strip())
        except ValueError:
            return np.nan

# Exemple d'utilisation :
print("Exemple clean_storage_capacity (MB):", clean_storage_capacity("512MB"))
print("Exemple clean_storage_capacity (GB):", clean_storage_capacity("16GB"))


### Fonction `clean_title`

**Description :**
La fonction `clean_title` nettoie le titre du produit en supprimant une liste de mots ou expressions non pertinents.

**Étapes :**
1. Définir une liste de mots à retirer (ex. "smart watch", "bluetooth", "2024", etc.).
2. Utiliser des expressions régulières pour supprimer ces mots du titre.
3. Nettoyer les espaces superflus et retourner le titre final.


In [ ]:
def clean_title(title):
    words_to_remove = [
        "smart watch", "smartwatch", "watch", "fitness tracker", "activity tracker", "sports watch", "wristwatch",
        "bluetooth", "gps", "wifi", "heart rate monitor", "blood pressure monitor", "waterproof", "ip67", "ip68",
        "touch screen", "phone function", "sos", "sleep monitor", "pedometer", "for men", "for women", "men's",
        "women's", "kids", "unisex", "new", "2024", "2025", "latest", "original", "brand new", "used", "mm", "gb",
        "mah", "android", "ios", "lte", "cellular", "wifi", "bluetooth",
        "amoled", "display", "screen", "flashlight", "compass", "military", "Women", "Good", "Very", "Modes", "Black", "White", "Silver",
        "Gold", "Blue", "Red", "Green", "Pink", "Purple", "Yellow", "Orange", "Brown", "Gray", "Grey", "Beige", "Ivory", "Cream", "Copper", "Bronze",
        "Coral", "Turquoise", "Aqua", "Lavender", "Lilac", "Indigo",
        "Maroon", "Olive", "Mint", "Teal", "Navy", "Apricot", "Azure", "Lime", "Violet", "Peach", "Plum", "Tan", "Khaki", "Crimson", "Magenta",
        "Salmon", "Charcoal", "Mauve", "Fuchsia", "Watches", "Watch", "Smart", "Smartwatch", "Fitness", "Tracker", "Activity", "Sports",
        "Wristwatch", "Bluetooth", "Gps", "Wifi", "Heart", "Rate", "Monitor", "Blood", "Pressure", "Waterproof", "Ip67", "Ip68", "Touch",
        "Screen", "Phone", "Function", "Sos", "Sleep", "Pedometer"
    ]
    for word in words_to_remove:
        title = re.sub(rf'\b{word}\b', '', title, flags=re.IGNORECASE)
    title = re.sub(r'1st Gen', 'I', title)
    title = re.sub(r'\b[A-Z0-9]{5,}\b', '', title)
    title = re.sub(r'\s+', ' ', title).strip()
    return title

# Exemple d'utilisation :
print("Exemple clean_title:", clean_title("New Apple Smart Watch Series 6 - 1st Gen, Black, 42mm"))


### Fonction `remove_duplicates`

**Description :**
La fonction `remove_duplicates` supprime les doublons dans le DataFrame en se basant sur un sous-ensemble de colonnes, après avoir trié par prix.

**Étapes :**
1. Trier le DataFrame par la colonne du prix en ordre croissant.
2. Supprimer les doublons en conservant la première occurrence.


In [ ]:
def remove_duplicates(df, subset_columns, price_column='Price'):
    df = df.sort_values(by=price_column, ascending=True)
    df = df.drop_duplicates(subset=subset_columns, keep='first')
    return df

# Exemple d'utilisation :
example_df = pd.DataFrame({
    'Brand': ['Apple', 'Apple'],
    'Model': ['Series 6', 'Series 6'],
    'Price': [1000, 950]
})
print("Exemple remove_duplicates:")
print(remove_duplicates(example_df, ['Brand', 'Model']))


## Fonction principale `clean_smart_watches_ebay`

**Description :**
Cette fonction applique toutes les fonctions de nettoyage précédemment définies sur le DataFrame contenant les données eBay.
Les étapes réalisées sont :
1. Nettoyage de la colonne `Price` puis imputation des valeurs manquantes (médiane).
2. Nettoyage de la colonne `Case Size` puis imputation (moyenne).
3. Nettoyage de la colonne `Battery Capacity` puis imputation (moyenne) et conversion en entier.
4. Nettoyage de la colonne `Brand` à l’aide de `clean_brand`.
5. Nettoyage de la colonne `Model` avec `clean_model` (la liste des modèles est extraite du DataFrame).
6. Nettoyage de la colonne `Operating System` avec `clean_os`.
7. Nettoyage de la colonne `Storage Capacity` puis imputation (médiane).
8. Nettoyage de la colonne `Title` via `clean_title`.
9. Affichage des valeurs manquantes pour vérification.
10. Filtrage des lignes contenant trop d’éléments `"Unknown"`.
11. Suppression des doublons sur un sous-ensemble de colonnes.


In [ ]:
def clean_smart_watches_ebay(df):
    # Nettoyage du Prix
    df['Price'] = df['Price'].apply(clean_price)
    price_imputer = SimpleImputer(strategy='median')
    df['Price'] = price_imputer.fit_transform(df[['Price']])

    # Nettoyage de la Taille du Boîtier
    df['Case Size'] = df['Case Size'].apply(clean_case_size)
    case_size_imputer = SimpleImputer(strategy='mean')
    df['Case Size'] = case_size_imputer.fit_transform(df[['Case Size']])
    df['Case Size'] = df['Case Size'].round(2)

    # Nettoyage de la Capacité de la Batterie
    df['Battery Capacity'] = df['Battery Capacity'].apply(clean_battery_capacity)
    battery_capacity_imputer = SimpleImputer(strategy='mean')
    df['Battery Capacity'] = battery_capacity_imputer.fit_transform(df[['Battery Capacity']])
    df['Battery Capacity'] = df['Battery Capacity'].round(0).astype(int)

    # Nettoyage de la Marque
    df['Brand'] = df.apply(lambda row: clean_brand(row['Brand'], row['Title']), axis=1)

    # Nettoyage du Modèle
    models_list = list(df['Model'].dropna().unique())
    df['Model'] = df['Model'].apply(lambda m: clean_model(m, models_list))

    # Nettoyage du Système d'Exploitation
    df['Operating System'] = df['Operating System'].apply(clean_os)

    # Nettoyage de la Capacité de Stockage
    df['Storage Capacity'] = df['Storage Capacity'].apply(clean_storage_capacity)
    storage_capacity_imputer = SimpleImputer(strategy='median')
    df['Storage Capacity'] = storage_capacity_imputer.fit_transform(df[['Storage Capacity']])

    # Nettoyage du Titre
    df['Title'] = df['Title'].apply(clean_title)

    # Afficher le nombre de valeurs manquantes par colonne pour vérification
    print("Valeurs manquantes par colonne :\n", df.isnull().sum())

    # Filtrer les lignes avec moins de deux informations 'Unknown'
    df = df[df.apply(lambda row: list(row).count('Unknown') < 2, axis=1)]

    # Suppression des doublons
    subset_columns = ['Brand', 'Model', 'Storage Capacity', 'Case Size', 'Battery Capacity']
    df = remove_duplicates(df, subset_columns)

    return df


## Traitement des fichiers CSV eBay

**Description :**
Cette section parcourt tous les fichiers CSV dans le répertoire des données brutes, applique la fonction `clean_smart_watches_ebay` et sauvegarde les fichiers nettoyés dans le répertoire dédié.


In [ ]:
try:
    for file in RAW_DATA_DIR_EBAY.glob('*.csv'):
        print(f"Processing eBay file: {file}")
        df_ebay = pd.read_csv(file)
        cleaned_df_ebay = clean_smart_watches_ebay(df_ebay)
        output_filename = CLEANED_DATA_DIR_EBAY / f"{file.stem}_cleaned.csv"
        cleaned_df_ebay.to_csv(output_filename, index=False)
        print(f"Cleaned data saved to {output_filename}")
except Exception as e:
    print(f"An error occurred while processing eBay data: {e}")


## Conclusion

Ce notebook a présenté un pipeline complet pour le nettoyage des données des montres connectées issues d'eBay.
Chaque étape a été détaillée afin de faciliter la compréhension et la maintenance du code.

Vous pouvez désormais utiliser ces données nettoyées pour vos analyses ou pour le développement de modèles prédictifs.


In [1]:
import pandas as pd
from pathlib import Path

# Définir le chemin vers le fichier CSV nettoyé
csv_file_path = Path(r'C:\Users\AdMin\Desktop\ecommerce_scraper\data\cleaned\ebay\smart_watches\smart_watches_2025_01_29_scrape1_cleaned.csv')

df_cleaned = pd.read_csv(csv_file_path)

# Set the option to display all rows
pd.set_option('display.max_rows', None)

# Display all rows of the DataFrame
df_cleaned.head(10)

,Title,Price,Case Size,Battery Capacity,Brand,Model,Operating System,Storage Capacity,Collection Date
0,Apple Series 5 40mm Space Aluminum Case Only SEE,0.99,40.00,305,Apple,Apple Watch Series 5,Apple Watch OS,32.0,1/29/2025 22:25
1,"1.83"" ROHS Wireless 4.0 , Many Options 5.0, 9....",9.95,41.35,305,Unknown,Smart Watch,Android Wear,32.0,1/29/2025 22:25
2,Men Ladies for iPhone Samsung,13.05,44.00,220,Unknown,Smart Bracelet,Android Wear,32.0,1/29/2025 22:25
3,Touchscreen Wrist,15.90,44.00,305,Unknown,X6 Smart Watch,Android Wear,32.0,1/29/2025 22:24
4,"/, iPhone Samsung..",16.99,26.00,305,Unknown,1,Android Wear,32.0,1/29/2025 22:25
5,Apple Series 4 - 44mm - Nike,17.43,44.00,305,Apple,Apple Watch Series 4 Nike+,Unknown,32.0,1/29/2025 22:25
6,Itouch Air 4 Brand In Box,18.00,41.00,305,Unknown,iTouch Air 4,Apple Watch OS,32.0,1/29/2025 22:25
7,"/, iPhone Samsung",19.99,22.00,300,IOWODO,Smart Watches,Android Wear,1.0,1/29/2025 22:24
8,Men for Samsung,20.79,41.35,305,Unknown,Smart Bracelet,Android Wear,32.0,1/29/2025 22:24
9,Calling & – with Multi-Sport,23.99,50.00,305,Unknown,Smart Bracelet,Android Wear,32.0,1/29/2025 22:25
